In [26]:
pip install scikit-learn -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing Libraries

In [27]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from scipy.spatial import distance

Importing the dataset and spliting to train and test

In [3]:
train = pd.read_csv("trainKNN.txt")
train.columns = ['ID', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type of glass']
train = train.drop('ID', axis=1) 

test = pd.read_csv('testKNN.txt')
test.columns=['ID', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type of glass']
test = test.drop('ID', axis=1)

In [4]:
train.shape

(195, 10)

In [5]:
test.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
0,1.52152,13.12,3.58,0.90,72.20,0.23,9.82,0.0,0.16,1
1,1.52300,13.31,3.58,0.82,71.99,0.12,10.17,0.0,0.03,1
2,1.51709,13.00,3.47,1.79,72.72,0.66,8.18,0.0,0.00,2
3,1.51660,12.99,3.18,1.23,72.97,0.58,8.81,0.0,0.24,2
4,1.51839,12.85,3.67,1.24,72.57,0.62,8.68,0.0,0.35,2


Discribing to find errors

In [6]:
train.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
count,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000
mean,1.518281,13.373846,2.750103,1.456154,72.639795,0.521744,8.911538,0.165077,0.050513,2.676923
std,0.003057,0.784929,1.390655,0.492293,0.763064,0.673624,1.425102,0.486304,0.086506,2.064234
min,1.511310,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,1.516455,12.875000,2.335000,1.190000,72.330000,0.145000,8.220000,0.000000,0.000000,1.000000
50%,1.517630,13.270000,3.480000,1.360000,72.810000,0.560000,8.570000,0.000000,0.000000,2.000000
75%,1.518950,13.795000,3.605000,1.625000,73.080000,0.610000,9.105000,0.000000,0.090000,3.000000
max,1.533930,15.790000,3.980000,3.500000,75.180000,6.210000,16.190000,3.150000,0.340000,7.000000


In [7]:
test.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
count,17.000000,17.000000,17.000000,17.00000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000
mean,1.518991,13.805294,1.769412,1.37000,72.849412,0.257647,9.437647,0.310000,0.128235,4.176471
std,0.002810,1.101659,1.744772,0.58676,0.910415,0.272799,1.419386,0.631951,0.169567,2.157409
min,1.511150,12.850000,0.000000,0.34000,71.360000,0.000000,6.650000,0.000000,0.000000,1.000000
25%,1.517110,13.000000,0.000000,0.90000,72.200000,0.000000,8.620000,0.000000,0.000000,2.000000
50%,1.519340,13.380000,1.610000,1.40000,72.720000,0.160000,8.990000,0.000000,0.000000,5.000000
75%,1.520650,14.230000,3.540000,1.94000,73.390000,0.560000,10.170000,0.150000,0.240000,6.000000
max,1.523000,17.380000,3.780000,2.17000,75.410000,0.760000,12.500000,1.670000,0.510000,7.000000


Normalizing

In [8]:
def normalize (df):
    for col in df.columns:
        if col != "Type of glass": 
            df[col] = (df[col] - df[col].mean())/df[col].std()
    return df

In [9]:
train = normalize(train)
test = normalize(test)

In [10]:
train.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
0,-0.219548,0.65758,0.611149,-0.195318,0.118214,-0.061969,-0.758920,-0.339452,-0.583925,1
1,-0.687312,0.19894,0.575195,0.170317,0.458946,-0.195574,-0.794005,-0.339452,-0.583925,1
2,-0.203193,-0.20874,0.675867,-0.337510,-0.039046,0.071637,-0.485255,-0.339452,-0.583925,1
3,-0.281699,-0.13230,0.625531,-0.439075,0.576892,0.041947,-0.590511,-0.339452,-0.583925,1
4,-0.759276,-0.74382,0.618340,0.332822,0.432736,0.175553,-0.590511,-0.339452,2.421657,1


In [11]:
test.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
0,0.900154,-0.622057,1.037722,-0.801009,-0.713314,-0.101346,0.269379,-0.490544,0.187328,1
1,1.426849,-0.449590,1.037722,-0.937351,-0.943978,-0.504574,0.515965,-0.490544,-0.579331,1
2,-0.676372,-0.730983,0.974676,0.715795,-0.142146,1.474908,-0.886050,-0.490544,-0.756252,2
3,-0.850750,-0.740061,0.808465,-0.238598,0.132454,1.181651,-0.442196,-0.490544,0.659119,2
4,-0.213734,-0.867142,1.089304,-0.221556,-0.306906,1.328280,-0.533785,-0.490544,1.307830,2


In [20]:
for i in [1,2,3,4,5,6,7,8,9,10]:
  euclid_model = KNeighborsClassifier(n_neighbors=i, metric=distance.sqeuclidean)
  manhattan_model = KNeighborsClassifier(n_neighbors=i, metric=distance.cityblock) 

  x_train = train.drop(["Type of glass"], axis=1)
  y_train = train["Type of glass"]

  euclid_model.fit(x_train,y_train)
  manhattan_model.fit(x_train, y_train)

  x_test = test.drop("Type of glass", axis=1) 
  y_test = test["Type of glass"]

  print("K value for Euclid  : " , i, " score : ", np.mean(cross_val_score(euclid_model, x_train, y_train, cv=6)))
  print("K value for manhattan : " , i, " score : ", np.mean(cross_val_score(manhattan_model, x_train, y_train, cv=6)))
  print()

K value for Euclid  :  1  score :  0.6253156565656566
K value for manhattan :  1  score :  0.6657196969696969

K value for Euclid  :  2  score :  0.69760101010101
K value for manhattan :  2  score :  0.6717171717171717

K value for Euclid  :  3  score :  0.66635101010101
K value for manhattan :  3  score :  0.6766098484848485

K value for Euclid  :  4  score :  0.6870265151515151
K value for manhattan :  4  score :  0.6816603535353535

K value for Euclid  :  5  score :  0.6871843434343434
K value for manhattan :  5  score :  0.6661931818181818

K value for Euclid  :  6  score :  0.6923926767676768
K value for manhattan :  6  score :  0.6816603535353535

K value for Euclid  :  7  score :  0.6614583333333334
K value for manhattan :  7  score :  0.6666666666666666

K value for Euclid  :  8  score :  0.6611426767676768
K value for manhattan :  8  score :  0.7024936868686869

K value for Euclid  :  9  score :  0.6515151515151515
K value for manhattan :  9  score :  0.6822916666666666

K val

K value of 6 has the better score among the other values

In [29]:
euclid_model = KNeighborsClassifier(n_neighbors=6, metric=distance.sqeuclidean) 
manhattan_model = KNeighborsClassifier(n_neighbors=6, metric=distance.cityblock) 
x_train = train.drop(["Type of glass"], axis=1)
y_train = train["Type of glass"]
euclid_model.fit(x_train,y_train) 
manhattan_model.fit(x_train, y_train)
x_test = test.drop("Type of glass", axis=1) 
y_test = test["Type of glass"]

Pridicting the values.

In [30]:
manhattan_predictions = manhattan_model.predict(x_test)
euclid_predictions = euclid_model.predict(x_test) 
df = pd.DataFrame({'actual': y_test, 'manhattan': manhattan_predictions, 'euclid': euclid_predictions})
df.head()


,actual,manhattan,euclid
0,1,1,1
1,1,1,1
2,2,2,2
3,2,1,1
4,2,1,2


Evaluating the Perfomance.

In [34]:
manhattan_count = len(df.loc[df['manhattan'] == df['actual']])
euclid_count = len(df.loc[df['euclid'] == df['actual']])

print('Manhattan Accuracy: {}%'.format(round(100*manhattan_count/len(df), 2)))
print(classification_report(y_test, manhattan_predictions, target_names=df['actual'].astype(str).unique()))
print ()

print('Square Euclidean Accuracy: {}%'.format(round(100*euclid_count/len(df), 2)))
print(classification_report(y_test, euclid_predictions, target_names=df['actual'].astype(str).unique()))

Manhattan Accuracy: 64.71%
              precision    recall  f1-score   support

           1       0.40      1.00      0.57         2
           2       0.25      0.33      0.29         3
           3       0.00      0.00      0.00         3
           5       1.00      0.67      0.80         3
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         3

    accuracy                           0.65        17
   macro avg       0.61      0.67      0.61        17
weighted avg       0.62      0.65      0.61        17


Square Euclidean Accuracy: 58.82%
              precision    recall  f1-score   support

           1       0.50      1.00      0.67         2
           2       0.29      0.67      0.40         3
           3       0.00      0.00      0.00         3
           5       1.00      0.67      0.80         3
           6       1.00      0.33      0.50         3
           7       1.00      1.00      1.00         3

    accuracy   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat